# Training of Recurrenct Neural Network (RNN) model on spectrograms

In [19]:
import numpy as np
import pandas as pd

from scipy import signal
from scipy.io import wavfile
import matplotlib.pyplot as plt

from PIL import Image
import matplotlib
import random

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras import layers


tf.config.run_functions_eagerly

<function tensorflow.python.eager.polymorphic_function.polymorphic_function.run_functions_eagerly(run_eagerly)>

In [55]:
# Loading of datasets

spectr_train_list = pd.read_csv("spectr_train_list.txt", index_col = 0).iloc[:,0].values.tolist()
spectr_test_list = pd.read_csv("spectr_test_list.txt", index_col = 0).iloc[:,0].values.tolist()

# labels_train, labels_test
# spectrograms_train, spectrograms_test

labels_train = []
spectrograms_train = []

for path in spectr_train_list:
    labels_train.append(path.split("/")[1])
    
    
    rgb = matplotlib.image.imread(path)
    gray = np.mean(rgb, -1)
    spectrogram = (255*(gray - np.min(gray))/np.ptp(gray)).astype(int)  # [0, 255] scaled

    spectrograms_train.append(spectrogram)

labels_test = []
spectrograms_test = []

for path in spectr_test_list:
    labels_test.append(path.split("/")[1])
    
    rgb = matplotlib.image.imread(path)
    gray = np.mean(rgb, -1)
    spectrogram = (255*(gray - np.min(gray))/np.ptp(gray)).astype(int)  # [0, 255] scaled
    
    spectrograms_test.append(spectrogram)


In [58]:
labels_test[0:9]

['no', 'up', 'off', 'go', 'yes', 'stop', 'down', 'on', 'down']

In [66]:
# preparing datasets for models

dummies = pd.get_dummies(labels_train)
y_train = tf.one_hot(dummies,10)

dataset_train = tf.data.Dataset.from_tensor_slices((spectrograms_train, dummies)).shuffle(200).batch(8)

####################

dummies2 = pd.get_dummies(labels_test)
y_test = tf.one_hot(dummies2, 10)

dataset_test = tf.data.Dataset.from_tensor_slices((spectrograms_test, dummies2)).batch(8)

#####################

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                           mode='min',
                           verbose=1,
                           patience=4)


# Model architecture

In [62]:
# model - LSTM
def get_model_RNN():
  model = tf.keras.models.Sequential()

  model.add(tf.keras.layers.LSTM(units = 64, return_sequences = True, input_shape=[64,64]))
  model.add(tf.keras.layers.LSTM(units = 64, return_sequences = True))

  model.add(tf.keras.layers.Dense(units = 32))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(units = 10, activation='softmax'))

  model.compile(loss = 'categorical_crossentropy',
             optimizer = keras.optimizers.Adam(learning_rate = 0.0001),
             metrics = ['accuracy'])
  return model

# Model training & ealuation

In [63]:
# model 1

model_RNN = get_model_RNN()

model_RNN.fit(dataset_train, epochs=50,  callbacks =[early_stop], validation_data=dataset_test)

Epoch 1/50
483/483 [==============================] - 32s 55ms/step - loss: 2.2395 - accuracy: 0.1606 - val_loss: 2.1018 - val_accuracy: 0.2665
Epoch 2/50
483/483 [==============================] - 25s 52ms/step - loss: 1.9484 - accuracy: 0.3098 - val_loss: 1.7458 - val_accuracy: 0.4033
Epoch 3/50
483/483 [==============================] - 25s 51ms/step - loss: 1.6072 - accuracy: 0.4389 - val_loss: 1.4957 - val_accuracy: 0.4822
Epoch 4/50
483/483 [==============================] - 24s 50ms/step - loss: 1.3887 - accuracy: 0.5109 - val_loss: 1.3376 - val_accuracy: 0.5482
Epoch 5/50
483/483 [==============================] - 24s 50ms/step - loss: 1.2319 - accuracy: 0.5736 - val_loss: 1.2266 - val_accuracy: 0.5769
Epoch 6/50
483/483 [==============================] - 25s 51ms/step - loss: 1.1074 - accuracy: 0.6197 - val_loss: 1.1362 - val_accuracy: 0.6148
Epoch 7/50
483/483 [==============================] - 25s 51ms/step - loss: 0.9966 - accuracy: 0.6554 - val_loss: 1.0613 - val_accuracy:

# Model 2 - more robust

In [64]:
# model - LSTM
def get_model_RNN2():
  model = tf.keras.models.Sequential()

  model.add(tf.keras.layers.LSTM(units = 128, return_sequences = True, input_shape=[64,64]))
  model.add(tf.keras.layers.LSTM(units = 128, return_sequences = True))

  model.add(tf.keras.layers.Dense(units = 64))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(units = 10, activation='softmax'))

  model.compile(loss = 'categorical_crossentropy',
             optimizer = keras.optimizers.Adam(learning_rate = 0.0001),
             metrics = ['accuracy'])
  return model

In [65]:
# model 2

model_RNN2 = get_model_RNN2()

model_RNN2.fit(dataset_train, epochs=50,  callbacks =[early_stop], validation_data=dataset_test)

Epoch 1/50
483/483 [==============================] - 49s 91ms/step - loss: 2.1186 - accuracy: 0.2306 - val_loss: 1.7894 - val_accuracy: 0.3892
Epoch 2/50
483/483 [==============================] - 43s 90ms/step - loss: 1.6071 - accuracy: 0.4293 - val_loss: 1.4027 - val_accuracy: 0.5237
Epoch 3/50
483/483 [==============================] - 45s 92ms/step - loss: 1.2763 - accuracy: 0.5459 - val_loss: 1.1683 - val_accuracy: 0.5891
Epoch 4/50
483/483 [==============================] - 45s 93ms/step - loss: 1.0340 - accuracy: 0.6324 - val_loss: 1.0353 - val_accuracy: 0.6452
Epoch 5/50
483/483 [==============================] - 44s 91ms/step - loss: 0.8422 - accuracy: 0.7000 - val_loss: 0.9232 - val_accuracy: 0.6739
Epoch 6/50
483/483 [==============================] - 45s 93ms/step - loss: 0.6917 - accuracy: 0.7518 - val_loss: 0.8720 - val_accuracy: 0.7089
Epoch 7/50
483/483 [==============================] - 44s 90ms/step - loss: 0.5552 - accuracy: 0.8070 - val_loss: 0.8621 - val_accuracy:

In [67]:
model_RNN2.fit(dataset_train, epochs=20,  callbacks =[early_stop], validation_data=dataset_test)

Epoch 1/20
483/483 [==============================] - 44s 91ms/step - loss: 0.1433 - accuracy: 0.9617 - val_loss: 0.9758 - val_accuracy: 0.7797
Epoch 2/20
483/483 [==============================] - 44s 90ms/step - loss: 0.0965 - accuracy: 0.9744 - val_loss: 1.1706 - val_accuracy: 0.7621
Epoch 3/20
483/483 [==============================] - 45s 93ms/step - loss: 0.0722 - accuracy: 0.9821 - val_loss: 1.1034 - val_accuracy: 0.7855
Epoch 4/20
483/483 [==============================] - 43s 89ms/step - loss: 0.0773 - accuracy: 0.9790 - val_loss: 1.0642 - val_accuracy: 0.7914
Epoch 5/20
483/483 [==============================] - 43s 89ms/step - loss: 0.0562 - accuracy: 0.9876 - val_loss: 1.1104 - val_accuracy: 0.7943
Epoch 6/20
483/483 [==============================] - 44s 90ms/step - loss: 0.0474 - accuracy: 0.9896 - val_loss: 1.1606 - val_accuracy: 0.7995
Epoch 6: early stopping


# Saving the model

In [68]:
model_RNN2.save('./model2.h5',save_format='h5')


In [69]:
model_RNN.save('./model.h5',save_format='h5')

In [70]:
model = tf.keras.models.load_model('./model2.h5')

# Conclusions

The first model obtained **74\% accuracy** on validation data set, whereas the second model obtained almost **80\% accuracy** with the same setting. 

Reminder: it was a 10 class classification problem with balanced classes.

Possible ideas to train better version of the final model:

- decreasing learning rate, as models started to overfit ( extended learning process)
- use more layers and units within each layer ( more robust models)
- prepare other version of spectrograms (better data means better model)